In [2]:
import sys
print('python version is',sys.version[:5])

python version is 3.5.2


In [4]:
import numpy as np
X = np.random.random((1000,3))

array([[ 0.15105211,  0.19339128,  0.95330227],
       [ 0.03934564,  0.73280227,  0.24623281],
       [ 0.72529761,  0.29308545,  0.60418675],
       ..., 
       [ 0.07750004,  0.70606253,  0.98744051],
       [ 0.03282066,  0.38315149,  0.25053962],
       [ 0.20922124,  0.35414302,  0.50116732]])

In [11]:
def pairwise_numpy(X):
    return np.sqrt(((X[:, None, :] - X) ** 2).sum(-1)) # -1 means all axis
%timeit pairwise_numpy(X)

10 loops, best of 3: 56.4 ms per loop


In [10]:
X[:, None, :].shape, X.shape, (X[:, None, :] - X).shape

((1000, 1, 3), (1000, 3), (1000, 1000, 3))

In [13]:
# Pure python function
def pairwise_python(X):
    M = X.shape[0]
    N = X.shape[1]
    D = np.empty((M, M), dtype=np.float)
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)
            
    return D
%timeit pairwise_python(X)

1 loop, best of 3: 4.62 s per loop


In [20]:
from numba import double
from numba.decorators import jit, autojit

pairwise_numba = autojit(pairwise_python)

%timeit pairwise_numba(X)

The slowest run took 144.41 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 4.59 ms per loop


# Wow, 1000 times!!! numba improved computation time a lot by such a simple method

# Next we try Cython

In [34]:
%load_ext Cython


In [35]:
%%cython

import numpy as np
cimport cython
from libc.math cimport sqrt

@cython.boundscheck(False)
@cython.wraparound(False)
def pairwise_cython(double[:, ::1] X):
    cdef int M = X.shape[0]
    cdef int N = X.shape[1]
    cdef double tmp, d
    cdef double[:, ::1] D = np.empty((M,M), dtype=np.float64)
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i,k] - X[j,k]
                d += tmp * tmp
            D[i,j] = sqrt(d)
    return np.asarray(D)


In [36]:
%timeit pairwise_cython(X)

The slowest run took 6.45 times longer than the fastest. This could mean that an intermediate result is being cached.
10 loops, best of 3: 5.73 ms per loop
